In [1]:
args_pre = {
    'det_config': 'demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py',
    'det_checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth',
    'pose_detector_config': 'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w48_coco_256x192.py',
    'pose_detector_checkpoint': 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth',
    'pose_lifter_config': 'configs/body/3d_kpt_sview_rgb_vid/video_pose_lift/h36m/videopose3d_h36m_243frames_fullconv_supervised.py',
    'pose_lifter_checkpoint': '/home/ubuntu/epoch_80.pth',
    # Flags/Optional
#     'video_path': 'demo/resources/jeldwen-1.mp4',
    'rebase_keypoint_height': True,
    'norm_pose_2d': None,
    'num_instances': -1,
    'show': False,
    'out_video_root': 'vis_results',
    'device': 'cuda:0',
    'det_cat_id': 1,
    'bbox_thr': 0.9,
    'kpt_thr': 0.3,
    'use_oks_tracking': None,
    'tracking_thr': 0.3,
    'euro': None,
    'radius': 8,
    'thickness': 2,
    'detections_2d': ''
}
from types import SimpleNamespace
args = SimpleNamespace(**args_pre)

In [2]:
from demo import zach_v_body3d_two_stage_video as m_2d_3d_model 

In [3]:
# Test one video
import time
args.video_name = 'BAR-S_water_ballon'
args.file_path = 'demo/resources/test_vids/BAR-S_water_ballon.mov'
args.detections_2d = ""
# args.file_path = 'demo/resources/test_vids/gHO_sBM_c08_d20_mHO0_ch10.mp4'
t1 = time.time()
m_2d_3d_model.process_video(args)
print(f"Total time: {time.time() - t1}")

Stage #1: 2D pose detection.


Initialized Model
Video len: 1513


[TensorRT] ERROR: Parameter check failed at: engine.cpp::setBindingDimensions::1136, condition: profileMaxDims.d[i] >= dimensions.d[i]
[TensorRT] ERROR: safeContext.cpp (184) - Cudnn Error in configure: 7 (CUDNN_STATUS_MAPPING_ERROR)
[TensorRT] ERROR: FAILED_EXECUTION: std::exception
[TensorRT] ERROR: Parameter check failed at: engine.cpp::setBindingDimensions::1136, condition: profileMaxDims.d[i] >= dimensions.d[i]
[TensorRT] ERROR: safeContext.cpp (184) - Cudnn Error in configure: 7 (CUDNN_STATUS_MAPPING_ERROR)
[TensorRT] ERROR: FAILED_EXECUTION: std::exception
[TensorRT] ERROR: Parameter check failed at: engine.cpp::setBindingDimensions::1136, condition: profileMaxDims.d[i] >= dimensions.d[i]
[TensorRT] ERROR: safeContext.cpp (184) - Cudnn Error in configure: 7 (CUDNN_STATUS_MAPPING_ERROR)
[TensorRT] ERROR: FAILED_EXECUTION: std::exception
[TensorRT] ERROR: Parameter check failed at: engine.cpp::setBindingDimensions::1136, condition: profileMaxDims.d[i] >= dimensions.d[i]
[TensorRT]

Time: 0.3916623592376709
Time: 0.024144887924194336
Time: 0.027008771896362305
Time: 0.029262304306030273
Time: 0.03151750564575195
Time: 0.035668373107910156


[TensorRT] ERROR: Parameter check failed at: engine.cpp::setBindingDimensions::1136, condition: profileMaxDims.d[i] >= dimensions.d[i]
[TensorRT] ERROR: safeContext.cpp (184) - Cudnn Error in configure: 7 (CUDNN_STATUS_MAPPING_ERROR)
[TensorRT] ERROR: FAILED_EXECUTION: std::exception
[TensorRT] ERROR: Parameter check failed at: engine.cpp::setBindingDimensions::1136, condition: profileMaxDims.d[i] >= dimensions.d[i]
[TensorRT] ERROR: safeContext.cpp (184) - Cudnn Error in configure: 7 (CUDNN_STATUS_MAPPING_ERROR)
[TensorRT] ERROR: FAILED_EXECUTION: std::exception
[TensorRT] ERROR: Parameter check failed at: engine.cpp::setBindingDimensions::1136, condition: profileMaxDims.d[i] >= dimensions.d[i]
[TensorRT] ERROR: safeContext.cpp (184) - Cudnn Error in configure: 7 (CUDNN_STATUS_MAPPING_ERROR)
[TensorRT] ERROR: FAILED_EXECUTION: std::exception
[TensorRT] ERROR: Parameter check failed at: engine.cpp::setBindingDimensions::1136, condition: profileMaxDims.d[i] >= dimensions.d[i]
[TensorRT]

...

Time: 0.04298520088195801
Time: 0.05190324783325195
Time: 0.0457453727722168
Time: 0.04376816749572754


[TensorRT] ERROR: Parameter check failed at: engine.cpp::setBindingDimensions::1136, condition: profileMaxDims.d[i] >= dimensions.d[i]
[TensorRT] ERROR: safeContext.cpp (184) - Cudnn Error in configure: 7 (CUDNN_STATUS_MAPPING_ERROR)
[TensorRT] ERROR: FAILED_EXECUTION: std::exception
[TensorRT] ERROR: Parameter check failed at: engine.cpp::setBindingDimensions::1136, condition: profileMaxDims.d[i] >= dimensions.d[i]
[TensorRT] ERROR: safeContext.cpp (184) - Cudnn Error in configure: 7 (CUDNN_STATUS_MAPPING_ERROR)
[TensorRT] ERROR: FAILED_EXECUTION: std::exception
[TensorRT] ERROR: Parameter check failed at: engine.cpp::setBindingDimensions::1136, condition: profileMaxDims.d[i] >= dimensions.d[i]
[TensorRT] ERROR: safeContext.cpp (184) - Cudnn Error in configure: 7 (CUDNN_STATUS_MAPPING_ERROR)
[TensorRT] ERROR: FAILED_EXECUTION: std::exception
[TensorRT] ERROR: Parameter check failed at: engine.cpp::setBindingDimensions::1136, condition: profileMaxDims.d[i] >= dimensions.d[i]
[TensorRT]

Time: 0.04992389678955078
Time: 0.07387733459472656
Time: 0.04847002029418945
Time: 0.04959702491760254
Stage 2: 2D-to-3D pose lifting.


[TensorRT] ERROR: Parameter check failed at: engine.cpp::setBindingDimensions::1136, condition: profileMaxDims.d[i] >= dimensions.d[i]
[TensorRT] ERROR: safeContext.cpp (184) - Cudnn Error in configure: 7 (CUDNN_STATUS_MAPPING_ERROR)
[TensorRT] ERROR: FAILED_EXECUTION: std::exception
[TensorRT] ERROR: Parameter check failed at: engine.cpp::setBindingDimensions::1136, condition: profileMaxDims.d[i] >= dimensions.d[i]
[TensorRT] ERROR: safeContext.cpp (184) - Cudnn Error in configure: 7 (CUDNN_STATUS_MAPPING_ERROR)
[TensorRT] ERROR: FAILED_EXECUTION: std::exception
[TensorRT] ERROR: Parameter check failed at: engine.cpp::setBindingDimensions::1136, condition: profileMaxDims.d[i] >= dimensions.d[i]
[TensorRT] ERROR: safeContext.cpp (184) - Cudnn Error in configure: 7 (CUDNN_STATUS_MAPPING_ERROR)
[TensorRT] ERROR: FAILED_EXECUTION: std::exception


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 1513/1513, 1128.3 task/s, elapsed: 1s, ETA:     0s
Video "BAR-S_water_ballon" processed.
Total time: 101.3261661529541


In [ ]:
import glob
import re

video_directory = 'evaluation_vids'

def get_video_list():
    '''Get list of videos to process'''
    file_paths = glob.glob("demo/resources/{}/*".format(video_directory))
    video_names = []
    
    for path in file_paths:
        video_names.append(
            SimpleNamespace(
                **{
                    'video_name': re.search('/.*/.*/(.*)(?:[.])', path).group(1),
                    'file_path': 'demo/resources/{}/'.format(video_directory) + re.search('/.*/.*/(.*)', path).group(1)
                }
            )
        )
    
    return video_names


In [ ]:
def process_videos():
    '''Iterate through videos and process'''
    video_list = get_video_list()
    print(video_list)
    for video in video_list:
        args.video_name = video.video_name
        args.file_path = video.file_path
        m_2d_3d_model.process_video(args)
        
process_videos()

# Loading wrnch


In [ ]:
import json
import numpy as np

filename = 'work_dirs/tumeke_testing/wrnch_jsons/AFG_subject_switching_close_nurses_subjects1_partial.json'
f = open(filename, "r")
data = json.load(f)
f.close()

In [ ]:
def get_wrnch_raw_data():
    standard_format_data = []
    wrnch_to_hrnet = [16, 19, 17, 20, 18, 13, 12, 14, 11, 15, 10, 3, 2, 4, 1, 5, 0]
    frame_width = 720 # Update AUTOMATICALLY!
    frame_height = 1280 # Update AUTOMATICALLY!
    for i in range(len(data["frames"])):
        wrnch_frame = data["frames"][i]
        single_frame = []
        for single_person_data in wrnch_frame["persons"]:
            if ("pose2d" not in single_person_data):
                continue
            if ("joints" not in single_person_data["pose2d"]):
                continue
            arr_2d = single_person_data["pose2d"]["joints"]
            if (arr_2d == []):
                continue
            kpts2d = np.array(arr_2d).reshape((25, 2))
            kpts2d[:, 0] *= frame_width
            kpts2d[:, 1] *= frame_height
            kpts2d = np.where(kpts2d < 0, 0, kpts2d)

            standard_format = np.hstack((kpts2d[wrnch_to_hrnet], np.ones((17, 1))))
            single_frame.append({
                "keypoints": standard_format
            })
        standard_format_data.append(single_frame)
    
    return standard_format_data

wrnch_raw_data = get_wrnch_raw_data()

In [ ]:
# wrnch_raw_data[2]

In [ ]:
wrnch_df = raw_data_to_dataframe(wrnch_raw_data)
wrnch_df

# Load Hrnet 

In [ ]:
# Test loading pickeled keypoints
import pickle
with open ('work_dirs/tumeke_testing/pickle_files/AFG_subject_switching_close_nurses_subjects1_partial.p', 'rb') as fp:
    raw_data = pickle.load(fp)

'''Load picked data into numpy.
    
Bounding box values are: 'xyxy' = (left, top, right, bottom)

Key Point values are: (ndarray[Kx3]): x, y, score

'''    


In [ ]:
#Just focus on videos with one subject for now

raw_data[400]

In [ ]:
# r_shoulder_i = joint_order.index('right_shoulder')
# r_elbow_i = joint_order.index('right_elbow')
# l_shoulder_i = joint_order.index('left_shoulder')
# l_elbow_i = joint_order.index('left_elbow')

# keypoint_array = raw_data[2][0]['keypoints']
# keypoint_array[r_shoulder_i], keypoint_array[r_elbow_i]
# # euclidean_dist(np.array([483.336, 456.679]), np.array([470.485, 504.871])) # 49.876016931988474
# euclidean_dist(keypoint_array[r_shoulder_i], keypoint_array[r_elbow_i]) # 49.876083

In [ ]:
# r_shoulder_i = joint_order.index('right_shoulder')
# r_elbow_i = joint_order.index('right_elbow')
# r_hip_i= joint_order.index('right_hip')
# l_shoulder_i = joint_order.index('left_shoulder')
# l_elbow_i = joint_order.index('left_elbow')
# l_hip_i = joint_order.index('left_hip')

# for subject in raw_data[906]:
#     print(subject)
#     keypoint_array = subject['keypoints']
#     r_se_ed = euclidean_dist(keypoint_array[r_shoulder_i], keypoint_array[r_elbow_i])
#     r_sh_ed = euclidean_dist(keypoint_array[r_shoulder_i], keypoint_array[r_hip_i])
#     l_se_ed = euclidean_dist(keypoint_array[l_shoulder_i], keypoint_array[l_elbow_i])
#     l_sh_ed = euclidean_dist(keypoint_array[l_shoulder_i], keypoint_array[l_hip_i])
#     print(np.array([r_se_ed, r_sh_ed, l_se_ed, l_sh_ed]).mean())


In [ ]:
# TEST SORT
# sorted_raw_data = []
# for frame in raw_data:
#     sorted_raw_data.append(sorted(frame, key=lambda x: closest_subject_heuristic(x['keypoints']), reverse=True))
    
# print(sorted_raw_data[906])


In [ ]:
# View histogram of subjects per frame
# subjects = [len(i) for i in raw_data]
# plt.hist(subjects, bins=200)

In [ ]:
import numpy as np
import pandas as pd
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

joint_order = [
    "nose", 
    "left_eye", 
    "right_eye", 
    "left_ear", 
    "right_ear", 
    "left_shoulder", 
    "right_shoulder", 
    "left_elbow", 
    "right_elbow", 
    "left_wrist", 
    "right_wrist", 
    "left_hip", 
    "right_hip", 
    "left_knee", 
    "right_knee", 
    "left_ankle", 
    "right_ankle" 
]

# def index_of(val, in_list):
#     try:
#         return in_list.index(val)
#     except ValueError:
#         return -1 

# # initializing points in
# # numpy arrays
# point1 = np.array((1, 2))
# point2 = np.array((2, 4))
 
# # calculating Euclidean distance
# # using linalg.norm()
# dist = np.linalg.norm(point1 - point2)
# dist

def euclidean_dist(p1, p2):
    # Euclidean distance
    return np.linalg.norm(p1 - p2)

def closest_subject_heuristic(keypoint_array):
    '''Get avg euclidean distance between right shoulder/elbow, left shoulder/elbow, right shoulder/hip and left shoulder/hip''' 
    r_shoulder_i = joint_order.index('right_shoulder')
    r_elbow_i = joint_order.index('right_elbow')
    r_hip_i= joint_order.index('right_hip')
    l_shoulder_i = joint_order.index('left_shoulder')
    l_elbow_i = joint_order.index('left_elbow')
    l_hip_i = joint_order.index('left_hip')
    
    r_se_ed = euclidean_dist(keypoint_array[r_shoulder_i], keypoint_array[r_elbow_i])
    r_sh_ed = euclidean_dist(keypoint_array[r_shoulder_i], keypoint_array[r_hip_i])
    l_se_ed = euclidean_dist(keypoint_array[l_shoulder_i], keypoint_array[l_elbow_i])
    l_sh_ed = euclidean_dist(keypoint_array[l_shoulder_i], keypoint_array[l_hip_i])
    return np.array([r_se_ed, r_sh_ed, l_se_ed, l_sh_ed]).mean()

def pull_from_dict(key, dict):
    try:
        return dict[key]
    except (KeyError, TypeError):
        if key == 'bbox':
            return np.full(5,np.NaN)
        else:
            return np.NaN

def extract_subject(subject_num, subjects):
    try:
        return subjects[subject_num-1]
    except IndexError:
        return None 

def raw_data_to_dataframe(raw_data):
    """ Load into dataframe"""
    
    # Order subjects in each frame by bbox size
    sorted_raw_data = []
    for frame in raw_data:
        sorted_raw_data.append(sorted(frame, key=lambda x: closest_subject_heuristic(x['keypoints']), reverse=True))

    # Get first identified subject in every frame
    first_subj = np.array([extract_subject(1, i) for i in sorted_raw_data])

    # Extract relevant fields to be set as columns
    bbox = np.array([pull_from_dict('bbox', i) for i in first_subj])
    area = np.array([pull_from_dict('area', i) for i in first_subj])
    track_id = np.array([pull_from_dict('track_id', i) for i in first_subj])

    # Structure Keypoints
    keypoints_array = []
    for frame in first_subj:
        
        #TODO(znoland):sanity check this and make sure no issues!
        # Keypoint order! - configs/_base_/datasets/h36m.py
        if not frame:
            # If no data for subject
            frame_keypoints = {}
            for index in range(17):
                frame_keypoints['j{}_x'.format(index)] = np.NaN
                frame_keypoints['j{}_y'.format(index)] = np.NaN
                frame_keypoints['j{}_score'.format(index)] = np.NaN
                frame_keypoints['j{}_l'.format(index)] = joint_order[index]
            keypoints_array.append(frame_keypoints)
            
        else:
            # If data for subject
            frame_keypoints = {}
            for index, keypoint in enumerate(frame['keypoints']):
                frame_keypoints['j{}_x'.format(index)] = keypoint[0]
                frame_keypoints['j{}_y'.format(index)] = keypoint[1]
                frame_keypoints['j{}_score'.format(index)] = keypoint[2]
                frame_keypoints['j{}_l'.format(index)] = joint_order[index]
            keypoints_array.append(frame_keypoints)

    # Create seperate dataframes (otherwise can't combine as >1-dimensional fields in pandas or numpy)
    bbox_s = pd.DataFrame(bbox, columns=['bbox_left', 'bbox_top', 'bbox_right', 'bbox_bottom', 'bbox_score'])
    area_s = pd.DataFrame(area, columns=['area'])
    track_id_s = pd.DataFrame(track_id, columns=['track_id'])
    keypoints_s = pd.DataFrame(keypoints_array)


    # Create combined DataFrame
    df = pd.concat([bbox_s,area_s,track_id_s,keypoints_s], axis=1)
    
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'id'})
    return df

    #TODO(znoland): Set index as a frame number column (e.g. video1_frame_num)
    #TODO(znoland): Add column with the name of the video?

    
df = raw_data_to_dataframe(raw_data)
pd.set_option("display.max_columns", None)
# # Skip every other row
# df = df.iloc[::2, :]
# df = df.set_index('id')
df

In [ ]:
# Create column names
score_column_names = ['bbox_score']
for i in range(17):
    score_column_names.append('j{}_score'.format(i))

pd.DataFrame(df[score_column_names].median()).T

### Visualize Scores - Hist

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.show()
sns.set(rc={'figure.figsize':(21,15)})

import matplotlib.pyplot as plt
n_rows=3
n_cols=6
# Create the subplots
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols)

# Set x axis limit (keep all charts on same axis)
for row in axes:
    for chart in row:
        chart.set_xlim(0,1) 

for i, column in enumerate(df[score_column_names].columns):
    ax = sns.histplot(df[column],ax=axes[i//n_cols,i%n_cols])
    if column == 'bbox_score':
        ax.set_title('bbox_score')
    else: 
        ax.set_title(joint_order[i-1]) # Currently has bbox as well, so need to offset title

In [ ]:
def f2d_skeleton_smoothness(df):
    ''' Get Euclidean distance '''
    
    # Build dict to accumulate calcs
    dict_2d_smth = {}
    for j in range(17):
            joint_field = 'j{}_euc_dist'.format(j)
            dict_2d_smth[joint_field] = []

    # Iterate through rows (Standard shift(-1) can't be used with this calc, so must iterate)
    for i in range(len(df)):
        for j in range(17):
            joint_field_x = 'j{}_x'.format(j)
            joint_field_y = 'j{}_y'.format(j)
            joint_field_score = 'j{}_score'.format(j)
            
            #TODO(znoland): return NaN when score below certain amount?
            
            try:
                dict_2d_smth['j{}_euc_dist'.format(j)].append(
                    euclidean_dist(
                        np.array([df.loc[i, joint_field_x], df.loc[i, joint_field_y]]), 
                        np.array([df.loc[i+1, joint_field_x], df.loc[i+1, joint_field_y]])
                    )
                )
            except:
                print('skiped row {}'.format(i))

    return pd.DataFrame(dict_2d_smth)

df_2d_smth = f2d_skeleton_smoothness(df)
df_2d_smth


In [ ]:
# Median distance between joints and previous frame by joint
df_euc_by_joint = pd.DataFrame(df_2d_smth.median()).T
df_euc_by_joint.columns = joint_order
df_euc_by_joint.T

### Visualize Sample Euclidean Distance - Hist

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.show()
sns.set(rc={'figure.figsize':(21,15)})

# df_2d_smth.j0_euc_dist.hist()
# sns.histplot(df_2d_smth).show()
    
import matplotlib.pyplot as plt
n_rows=3
n_cols=6
# Create the subplots
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols)

# Set x axis limit (keep all charts on same axis)
for row in axes:
    for chart in row:
        chart.set_xlim(0,df_2d_smth.quantile(0.90).max()) # df_2d_smth.max().max()

for i, column in enumerate(df_2d_smth.columns):
    ax = sns.histplot(df_2d_smth[column],ax=axes[i//n_cols,i%n_cols])
    ax.set_title(joint_order[i]) 


## Load example ground truth video

In [ ]:
import json

with open('work_dirs/tumeke_testing/ground_truth_labels/AFG_subject_switching_close_nurses_subjects1_partial.json') as f:
  labels = json.load(f)

In [ ]:
len(labels)

In [ ]:
labels[0]

In [ ]:
# test = pd.DataFrame(labels[0]['pose-keypoints'])

# test['x_px'] = (test['x']) / 100.0 * test['original_width']
# test['y_px'] = (test['y']) / 100.0 * test['original_height']

# test

In [ ]:
# from matplotlib.pyplot import figure
# import matplotlib.image as mpimg 
# from scipy import ndimage

# img = mpimg.imread('work_dirs/tumeke_testing/ground_truth_images/BAR-S_water_ballon/00001274.jpg')
# sns.scatterplot(data=test, x="x_px", y="y_px")
# # ax.invert_yaxis()
# # width_perc = 720 / 1280
# # plt.rcParams["figure.figsize"] = (10 * width_perc, 10)
# # plt.xlim(0, 720)
# # plt.ylim(0, 1280)
# plt.imshow(img)
# plt.show()

## Structure ground truth data

In [ ]:
# img, id, keypoints (j0_x, j0_y)

# 1) loop through each frame
# 2) take the first person (wait till we have the relationships)
# 3) pivot into dataframe like previous function for ETL

In [ ]:
labels[0:1]

In [ ]:
import re

joint_order = [
    'Nose', 
    'L_Eye', 
    'R_Eye', 
    'L_Ear', 
    'R_Ear', 
    'L_Shoulder', 
    'R_Shoulder', 
    'L_Elbow', 
    'R_Elbow', 
    'L_Wrist', 
    'R_Wrist', 
    'L_Hip', 
    'R_Hip', 
    'L_Knee', 
    'R_Knee', 
    'L_Ankle', 
    'R_Ankle' 
]

def extract_x_and_y(joint):
    return np.array([joint['x'], joint['y']])


def closest_subject_heuristic_raw(keypoint_obj):
    '''Get avg euclidean distance between right shoulder/elbow, left shoulder/elbow, right shoulder/hip and left shoulder/hip''' 
    r_shoulder_i = joint_order.index('R_Shoulder')
    r_elbow_i = joint_order.index('R_Elbow')
    r_hip_i= joint_order.index('R_Hip')
    l_shoulder_i = joint_order.index('L_Shoulder')
    l_elbow_i = joint_order.index('L_Elbow')
    l_hip_i = joint_order.index('L_Hip')
    
    r_se_ed = euclidean_dist(extract_x_and_y(keypoint_obj[r_shoulder_i]), extract_x_and_y(keypoint_obj[r_elbow_i]))
    r_sh_ed = euclidean_dist(extract_x_and_y(keypoint_obj[r_shoulder_i]), extract_x_and_y(keypoint_obj[r_hip_i]))
    l_se_ed = euclidean_dist(extract_x_and_y(keypoint_obj[l_shoulder_i]), extract_x_and_y(keypoint_obj[l_elbow_i]))
    l_sh_ed = euclidean_dist(extract_x_and_y(keypoint_obj[l_shoulder_i]), extract_x_and_y(keypoint_obj[l_hip_i]))
    return np.array([r_se_ed, r_sh_ed, l_se_ed, l_sh_ed]).mean()


def ground_truth_processing(labels):
    '''Tranform ground truth data into data frame

        1) Get Subjects

        2) Get Keypoints for each subject

        3) Order subjects by heuristic

    '''

    frame_array = []

    for label in labels:
    #     label = label_r[0]
        # label = labels[0:1][0]

        row = {}

        # Metadata
        img_name = re.search(r'(\d\d+)', label['data']['img']).group(0)
        row['id'] = int(img_name)
        row['ls_id'] = label['id']
        row['img'] = label['data']['img']
        row['subjects'] = []

        #Subjects
        persons_t = [p for p in label['annotations'][0]['result'] if p['type'] == 'rectanglelabels']

        # Keypoints
        keypoints_t = [k for k in label['annotations'][0]['result'] if k['type'] == 'keypointlabels']

        for i, person in enumerate(persons_t):

            # Filter on subject's keypoints
            person_keypoints_t = [k for k in keypoints_t if k['parentID'] == person['id']]
            # Filter out unused keypoints
            person_keypoints_t = [k for k in person_keypoints_t if k['value']['keypointlabels'][0] in joint_order]

             # Set order of keypoints + set placeholders when null
            person_keypoints_sorted = []
            for i, joint_name in enumerate(joint_order):
                target_keypoint = [k for k in person_keypoints_t if k['value']['keypointlabels'][0] == joint_name]

                if target_keypoint:
                    original_width = target_keypoint[0]['original_width']
                    original_height = target_keypoint[0]['original_height']
                    target_keypoint = target_keypoint[0]['value']
                    target_keypoint['original_width'] = original_width
                    target_keypoint['original_height'] = original_height
                else:
                    target_keypoint = {'x': np.nan, 'y': np.nan, 'width': np.nan, 'keypointlabels': [joint_name], 'original_width': np.nan, 'original_height': np.nan, }

                person_keypoints_sorted.append(target_keypoint)

            subject_t = {
                'name': person['value']['rectanglelabels'][0],
                'id': person['id'],
                'pose-keypoints': person_keypoints_sorted,
                'closest-subject-heuristic': closest_subject_heuristic_raw(person_keypoints_sorted)
            }
            row['subjects'].append(subject_t)


        # Sort subjects by heuristic
        row['subjects'] = sorted(row['subjects'], key=lambda x: x['closest-subject-heuristic'], reverse=True)

        # Format x & y coordinates (for 1st subject only)
        for index, keypoint in enumerate(row['subjects'][0]['pose-keypoints']):
            if len(keypoint['keypointlabels']) > 1:
                raise ValueError('There are multiple labels for one keypoint!') 
            row['j{}_x'.format(index)] = keypoint['x'] / 100.0 * keypoint['original_width']
            row['j{}_y'.format(index)] = keypoint['y'] / 100.0 * keypoint['original_height']
            row['j{}_l'.format(index)] = keypoint['keypointlabels'][0]

        frame_array.append(row)
        
    return frame_array

# raw array is "frame_array" !
gt_frame_array = ground_truth_processing(labels)
gt_df = pd.DataFrame(gt_frame_array)
# Order by ID
gt_df = gt_df.sort_values(by='id')
gt_df = gt_df.reset_index(drop=True)
gt_df = gt_df.set_index('id')
gt_df

In [ ]:
gt_df['subjects'].loc[[2]].values[0]

In [ ]:
# OLDER METHOD FOR SIGNLE SUBJECT VIDEOS

# import re

# joint_order = [
#     'Nose', 
#     'L_Eye', 
#     'R_Eye', 
#     'L_Ear', 
#     'R_Ear', 
#     'L_Shoulder', 
#     'R_Shoulder', 
#     'L_Elbow', 
#     'R_Elbow', 
#     'L_Wrist', 
#     'R_Wrist', 
#     'L_Hip', 
#     'R_Hip', 
#     'L_Knee', 
#     'R_Knee', 
#     'L_Ankle', 
#     'R_Ankle' 
# ]

# def ground_truth_etl(labels):
#     '''Tranform ground truth data into data frame'''
#     frame_array = []
#     for label in labels:
#         row = {}
#         img_name = re.search(r'(\d\d+)', label['img']).group(0)
#         row['id'] = int(img_name)
#         row['ls_id'] = label['id']
#         row['img'] = label['img']
        
#         # Filter out unused keypoints
#         label['pose-keypoints'] = [keypoint for keypoint in label['pose-keypoints'] if keypoint['keypointlabels'][0] in joint_order]
#         # Set order of keypoints
#         label['pose-keypoints'].sort(key=lambda x: joint_order.index(x['keypointlabels'][0]))

#         for index, keypoint in enumerate(label['pose-keypoints']):
#             if len(keypoint['keypointlabels']) > 1:
#                 raise ValueError('There are multiple labels for one keypoint!') 
#             row['j{}_x'.format(index)] = keypoint['x'] / 100.0 * keypoint['original_width']
#             row['j{}_y'.format(index)] = keypoint['y'] / 100.0 * keypoint['original_height']
#             row['j{}_l'.format(index)] = keypoint['keypointlabels'][0]

#         frame_array.append(row)
        
#     return pd.DataFrame(frame_array)
        
# gt_df = ground_truth_etl(labels)
# # Order by ID
# gt_df = gt_df.sort_values(by='id')
# gt_df = gt_df.reset_index(drop=True)
# gt_df = gt_df.set_index('id')

In [ ]:
#NOTE: IT APPEARS AS IF A ROW IS NOT RETURNED WHEN a frame is not labeled!!!

pd.set_option("display.max_columns", None)
gt_df

In [ ]:
from matplotlib.pyplot import figure
import matplotlib.image as mpimg 
from scipy import ndimage
import re

def visualize_gt_frame(frame_row):
    img_name = re.search(r'(\d+[.][jpg]+)', frame_row['img'].values[0]).group(0)
    img = mpimg.imread('work_dirs/tumeke_testing/ground_truth_images/BAR-S_water_ballon/{}'.format(img_name))
    for j in range(17): # 19 originally
        x = "j{}_x".format(j)
        y = "j{}_y".format(j)
        l = "j{}_l".format(j)
        p1 = sns.scatterplot(data=frame_row, x=x, y=y)
        # Add text besides each point
        p1.text(frame_row[x]+10, frame_row[y], 
             frame_row[l].values[0], horizontalalignment='left', 
             size=7, color='white', weight='regular')
    plt.imshow(img)
    plt.show()

In [ ]:
# visualize_gt_frame(gt_df[0:1])
# visualize_gt_frame(gt_df[20:21])

In [ ]:
# gt_df[0:1].img.values[0] # 1 indexed - The frame number is the image name

In [ ]:
def visualize_overlap(frame_number):
    gt_f = gt_df.loc[[frame_number]]
    f = df.loc[[frame_number]]
    w_f = wrnch_df.loc[[frame_number]]
    
    frame_img_name = re.search(r'(\d+[.][jpg]+)', gt_f['img'].values[0]).group(0)
    file_img_name = re.findall(r'/([0-9a-zA-Z-_]+)', gt_df[0:1].img.values[0])[1]
    img = mpimg.imread('work_dirs/tumeke_testing/ground_truth_images/{}/{}'.format(file_img_name, frame_img_name))
    
    for j in range(17):
        x = "j{}_x".format(j)
        y = "j{}_y".format(j)
        l = "j{}_l".format(j)
        # Ground Truth - BLUE
        p1 = sns.scatterplot(data=gt_f, x=x, y=y, color='blue')
        p1.text(gt_f[x]-70, gt_f[y], 
             gt_f[l].values[0], horizontalalignment='left', 
             size=7, color='blue', weight='regular')
        # Hrnet Model - RED
        p2 = sns.scatterplot(data=f, x=x, y=y, color='red')
        p2.text(f[x]+20, f[y]-1, 
             f[l].values[0], horizontalalignment='left', 
             size=7, color='red', weight='regular')
        
        # Wrnch - GREEN
        p3 = sns.scatterplot(data=w_f, x=x, y=y, color='green')
        p3.text(w_f[x]+20, w_f[y]+1, 
             w_f[l].values[0], horizontalalignment='left', 
             size=7, color='green', weight='regular')
        
        
        
    plt.imshow(img)
    plt.show()
    
    
visualize_overlap(1000) # 1000

In [ ]:
# test = df['j0_x'] / gt_df['j0_x']
# test.count()

In [ ]:
# df.loc[[600]]

## PCK - Percentage of Correct Key-points

Description: https://github.com/cbsudux/Human-Pose-Estimation-101#percentage-of-correct-parts---pcp


Percentage of Correct Key-points - PCK:
* Detected joint is considered correct if the distance between the predicted and the true joint is within a certain threshold (threshold varies)
* PCKh@0.5 is when the threshold = 50% of the head bone link
* PCK@0.2 == Distance between predicted and true joint < 0.2 * torso diameter
* Sometimes 150 mm is taken as the threshold
* Head, shoulder, Elbow, Wrist, Hip, Knee, Ankle → Keypoints
* PCK is used for 2D and 3D (PCK3D)
* Higher the better

In [ ]:
# pelvis is in the middle of l_hip and r_hip
# keypoints_new[0] = (keypoints[11] + keypoints[12]) / 2
# thorax is in the middle of l_shoulder and r_shoulder
# keypoints_new[8] = (keypoints[5] + keypoints[6]) / 2

In [ ]:
# IMPORTANT !!!!!!!!!!! - THIS IS A TEST - CHANGE BACK!
# df = wrnch_df
# df

In [ ]:
def extract_j_pos(f, v, joint_name):
    return f['j{}_{}'.format(joint_order.index(joint_name), v)]

def get_torso_diameter(x):
    return euclidean_dist(np.array([x['pelvis_x'], x['pelvis_y']]), np.array([x['thorax_x'], x['thorax_y']]))

for df_i in [df, gt_df]:
    
    df_i['pelvis_x'] = (extract_j_pos(df_i, 'x', 'L_Hip') + extract_j_pos(df_i, 'x', 'R_Hip')) / 2
    df_i['pelvis_y'] = (extract_j_pos(df_i, 'y', 'L_Hip') + extract_j_pos(df_i, 'y', 'R_Hip')) / 2
    df_i['thorax_x'] = (extract_j_pos(df_i, 'x', 'L_Shoulder') + extract_j_pos(df_i, 'x', 'R_Shoulder')) / 2
    df_i['thorax_y'] = (extract_j_pos(df_i, 'y', 'L_Shoulder') + extract_j_pos(df_i, 'y', 'R_Shoulder')) / 2
    
    df_i['torso_diameter'] = df_i.apply(lambda x: get_torso_diameter(x), axis=1)

In [ ]:
# 1) Iterate through keypoints and check 
#    - distance between predicted and true joint < 0.2 * torso diameter

# 2) for row, get jX_d (joint detected)

# 3) Get percent of all values detected (across all keypoints)

In [ ]:
pck_threshold_s = .20 * gt_df['torso_diameter']
pck_threshold_s

In [ ]:
def vect_duclidean_dist(df1, df2, cols=['x_coord','y_coord']):
    return np.linalg.norm(df1[cols].values - df2[cols].values, axis=1)

pck_df_raw = gt_df[[]] # Return just index
for j in range(17):
    x = "j{}_x".format(j)
    y = "j{}_y".format(j)
    pck_df_raw['{}_ed'.format(joint_order[j])] = vect_duclidean_dist(gt_df, df.iloc[gt_df.index], cols=[x,y])
    # pck_df['{}'.format(joint_order[j])] = vec_euclidean_dist(gt_df[[x,y]], df.iloc[gt_df.index][[x,y]])

pck_df_raw

In [ ]:
pck_df_scores = pck_df_raw.lt(pck_threshold_s, axis='index') # True if distance less than PCK threshold
pck_df_scores = pck_df_scores.applymap(lambda x: 1 if x == True else 0)
pck_df_scores

In [ ]:
# % Detected frames
pck_df_scores.sum(axis=1).apply(lambda x: 1 if x > 0 else 0).sum() / pck_df_scores.sum(axis=1).count()

In [ ]:
# Total (all frames)
pck_df_scores.sum(axis=1).sum() / pck_df_scores.count(axis=1).sum()

In [ ]:
# Total (detected frames)
pck_detected_df = pck_df_scores.sum(axis=1).apply(lambda x: True if x > 0 else False)
pck_df_scores[pck_detected_df].sum(axis=1).sum() / pck_df_scores[pck_detected_df].count(axis=1).sum()

In [ ]:
# By Body part
pck_df_scores.sum() / pck_df_scores.count()

In [ ]:
# by frame
# pd.DataFrame(pck_df_scores.sum(axis=1) / pck_df_scores.count(axis=1), columns=['PCK']).plot.area()
pd.DataFrame(pck_df_scores).plot.area(title='PCK over time by joint')

## PCP - Percentage of Correct Parts


https://github.com/cbsudux/Human-Pose-Estimation-101#percentage-of-correct-parts---pcp

### Percentage of Correct Parts - PCP
* A limb is considered detected and a correct part if the distance between the two predicted joint locations and the true limb joint locations is at most half of the limb length (PCP at 0.5 )
* Measures detection rate of limbs
* Cons - penalizes shorter limbs

Calculation:
* For a specific part, PCP = (No. of correct parts for entire dataset) / (No. of total parts for entire dataset)
* Take a dataset with 10 images and 1 pose per image. Each pose has 8 parts - ( upper arm, lower arm, upper leg, lower leg ) x2
* No of upper arms = 10 * 2 = 20
* No of lower arms = 20
* No of lower legs = No of upper legs = 20
* If upper arm is detected correct for 17 out of the 20 upper arms i.e 17 ( 10 right arms and 7 left) → PCP = 17/20 = 85%

Higher the better

In [ ]:
# 1) ground truth limb length

# 2) Euclidean distance between joints (already done) + avg between start joint and end joint euclidean distances

# 3) Check if limb ed is less than threshold (ground truth limb length * .5)

In [ ]:
'''Ground Truth Limb Length'''
limbs = [
    {'name':'L_upper_arm', 'joints': [joint_order.index('L_Shoulder'), joint_order.index('L_Elbow')]},
    {'name':'R_upper_arm', 'joints': [joint_order.index('R_Shoulder'), joint_order.index('R_Elbow')]},
    {'name':'L_lower_arm', 'joints': [joint_order.index('L_Elbow'), joint_order.index('L_Wrist')]},
    {'name':'R_lower_arm', 'joints': [joint_order.index('R_Elbow'), joint_order.index('R_Wrist')]},
    {'name':'L_upper_leg', 'joints': [joint_order.index('L_Hip'), joint_order.index('L_Knee')]},
    {'name':'R_upper_leg', 'joints': [joint_order.index('R_Hip'), joint_order.index('R_Knee')]},
    {'name':'L_lower_leg', 'joints': [joint_order.index('L_Knee'), joint_order.index('L_Ankle')]},
    {'name':'R_lower_leg', 'joints': [joint_order.index('R_Knee'), joint_order.index('R_Ankle')]}
]

# def get_limb_diameter(r):
#     return euclidean_dist(np.array([r[x1], r[y1]]), np.array([r[x2], r[y2]]))

pcp_gt_limb_length_df = gt_df[[]] # Return just index
for limb in limbs:
    # Get ground truth limb length
    j1 = limb['joints'][0] # joint 1 index
    j2 = limb['joints'][1] # joint 2 index
    x1 = "j{}_x".format(j1)
    y1 = "j{}_y".format(j1)
    x2 = "j{}_x".format(j2)
    y2 = "j{}_y".format(j2)
    pcp_gt_limb_length_df[limb['name']] = gt_df.apply(lambda r: euclidean_dist(np.array([r[x1], r[y1]]), np.array([r[x2], r[y2]])), axis=1)

pcp_gt_limb_length_df

In [ ]:
'''PCP Theshold'''
pcp_threshold_df = pcp_gt_limb_length_df * 0.50
pcp_threshold_df

In [ ]:
'''Limb Euclidean Dist'''
limbs = [
    {'name':'L_upper_arm', 'joints': ['L_Shoulder', 'L_Elbow']},
    {'name':'R_upper_arm', 'joints': ['R_Shoulder', 'R_Elbow']},
    {'name':'L_lower_arm', 'joints': ['L_Elbow', 'L_Wrist']},
    {'name':'R_lower_arm', 'joints': ['R_Elbow', 'R_Wrist']},
    {'name':'L_upper_leg', 'joints': ['L_Hip', 'L_Knee']},
    {'name':'R_upper_leg', 'joints': ['R_Hip', 'R_Knee']},
    {'name':'L_lower_leg', 'joints': ['L_Knee', 'L_Ankle']},
    {'name':'R_lower_leg', 'joints': ['R_Knee', 'R_Ankle']}
]

pcp_limb_ed_df = gt_df[[]] # Return just index
for limb in limbs:
    j1 = limb['joints'][0] # joint 1 name
    j2 = limb['joints'][1] # joint 2 name
    
    pcp_limb_ed_df[limb['name']] = pck_df_raw['{}_ed'.format(j1)] + pck_df_raw['{}_ed'.format(j2)] / 2
    
pcp_limb_ed_df

In [ ]:
# pcp_df_scores = pck_df_raw.lt(pck_threshold_s, axis='index') # True if distance less than PCP threshold
# pcp_df_scores = pck_df_scores.applymap(lambda x: 1 if x == True else 0)
# pcp_df_scores



In [ ]:
pcp_df_scores = pcp_limb_ed_df.lt(pcp_threshold_df)
pcp_df_scores = pcp_df_scores.applymap(lambda x: 1 if x == True else 0)
pcp_df_scores

In [ ]:
# % Detected frames
pcp_df_scores.sum(axis=1).apply(lambda x: 1 if x > 0 else 0).sum() / pcp_df_scores.sum(axis=1).count()

In [ ]:
# Total (all frames)
pcp_df_scores.sum(axis=1).sum() / pcp_df_scores.count(axis=1).sum()

In [ ]:
# Total (detected frames)
pcp_detected_df = pcp_df_scores.sum(axis=1).apply(lambda x: True if x > 0 else False)
pcp_df_scores[pcp_detected_df].sum(axis=1).sum() / pcp_df_scores[pcp_detected_df].count(axis=1).sum()

In [ ]:
# By Limb
pcp_df_scores.sum() / pcp_df_scores.count()

In [ ]:
# by Frame
pd.DataFrame(pcp_df_scores).plot.area(title='PCP over time by limb')

In [ ]:
# Update summary calcs!:
# 1) For frames where there is an output, how accurate is that output? (if entire row = 0, then exclude row in calculation)
# 2) Calculate number of detected frames (rows that have at least 1 value in threshold (row > 1))


# Tests with Modules

In [ ]:
# from tumeke_testing_modules import load_model_data, load_gt_data
# from tumeke_testing_modules.visualize_frame import visualize_overlap
# from tumeke_testing_modules.pck_calc import pck_calc
# from tumeke_testing_modules.pcp_calc import pcp_calc
# from tumeke_testing_modules.visualize_df_scores import visualize_df_scores
# from tumeke_testing_modules.smoothness_calc import smoothness_calc

In [ ]:
# # file = 'AFG_subject_switching_close_nurses_subjects1_partial'
# file = 'BAR-S_water_ballon_trimmed' # Need to export full annotation file, not partial

# hrnet_df, wrnch_df = load_model_data.get_hrnet_wrnch_dfs(file)
# gt_df = load_gt_data.get_gt_df(file)

In [ ]:
# visualize_df_scores(hrnet_df)

In [ ]:
# pck_calc(gt_df, hrnet_df, wrnch_df)

In [ ]:
# visualize_overlap(gt_df, hrnet_df, wrnch_df, 300)

In [ ]:
# pcp_calc(gt_df, hrnet_df, wrnch_df)

In [ ]:
# smoothness_calc(gt_df, hrnet_df, wrnch_df)